In [2]:
import os
import sys
import datetime
import pathlib
import pandas as pd
dir_path = pathlib.Path(os.path.abspath('')).resolve()
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *
print(dir_path)

/mnt/OpenFormat/vector_data


In [2]:
import os
import pyarrow.parquet as pq
import pyarrow.orc as po
import pyarrow as pa
import h5py
import numpy as np
import pyarrow as pa
import pandas as pd
import numpy as np
import zarr
import xarray as xr

import os

def get_size(path):
    if os.path.isfile(path):
        # If the path is a file, use os.path.getsize
        return os.path.getsize(path)
    elif os.path.isdir(path):
        # If the path is a directory, sum the sizes of all the files in it
        total = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for f in filenames:
                fp = os.path.join(dirpath, f)
                total += os.path.getsize(fp)
        return total
    else:
        # If the path is neither a file nor a directory
        print("The path is neither a file nor a directory")
        return None

# old ones before I clean up duplicates
# file_to_embed_col = {
#                     'nielsr--datacomp-small-with-embeddings-and-cluster-labels.parquet' : ['clip_l14_embedding'],
#                      'crumb--flan-t5-small-embed-refinedweb.parquet': ['encoding'],
#                      'crumb--flan-t5-large-embed-refinedweb.parquet': ['encoding'],
#                      'xwjzds--ag_news_embed_test.parquet': ['test'],
#                      'justinian336--salvadoran-news-embedded.parquet': ['embedding'],
#                      'davanstrien--ia_embedded.parquet': ['embeddings'],
#                      'dhmeltzer--ELI5_embedded.parquet': ['embeddings'],
#                      'sanjin7--embedding_dataset_distilbert_base_uncased_ad_subwords.parquet': ['mean_embedding'],
#                      'lsb--openwebtext-all-minilm-l6-v2-embedding.parquet': ['embedding'],
#                      'davanstrien--pytorchmodelmetadata_with_embeddings.parquet': ['embedding'],
#                      'llm-book.parquet': ['embeddings'],
#                      'tollefj.parquet': ['embedding'],
#                      'github-embeddings-doy.parquet': ['embeddings'],
#                      'cohere-en.parquet': ['emb'],
#                      'cohere-simple.parquet': ['emb'],
#                      'laion.parquet': ['text_embedding'],
#                      }
file_to_embed_col = {
    'cohere-simple': ['emb'],
    'crumb--flan_t5': ['encoding'],
    'davanstrien--ia_embedded': ['embeddings'],
    'davanstrien--pytorchmodelmetadata_with_embeddings': ['embedding'],
    'dhmeltzer--eli5': ['embeddings'],
    'justinian336--salvadoran-news-embedded': ['embedding'],
    'KaiserML--SemanticScholar': ['embedding'],
    'KShivendu--openai': ['openai'],
    'laion': ['text_embedding'],
    'llm-book': ['embeddings'],
    'lsb--minilm': ['embedding'],
    'lsb--openwebtext-all-minilm-l6-v2-embedding': ['embedding'],
    'LukeSajkowski--eco': ['embeddings'],
    'maiia--mcphrasy': ['embeddings'],
    'Multimodal-Fatima--StanfordCars_train_embeddings': ['vision_embeddings'],
    'nielsr--datacomp-small-with-embeddings-and-cluster-labels': ['clip_l14_embedding'],
    'rjac--all_the_news': ['embedding'],
    'sanjin7--embedding_dataset_distilbert_base_uncased_ad_subwords': ['mean_embedding'],
    # 'tollefj': ['embedding'],
    'victorych22--lamini': ['embeddings'],
    'vjain--tax_embedding': ['embedding'],
    'xwjzds--ag_news_embed_test': ['test'],
    
    'mushfirat--diffusiondb': ['embedding'], # https://www.kaggle.com/datasets/mushfirat/diffusiondb-metadata-with-prompt-embeddings
    'alpayariyak--math_embedded': ['embedding'], # https://huggingface.co/datasets/alpayariyak/MATH_Embedded_Instructor-XL/tree/main
    'milly233--gov_data': ['embedding'], # https://huggingface.co/datasets/milly233/govdatata_embedding/tree/main
    'CShorten--ArXivML': ['embedding'], # https://huggingface.co/datasets/CShorten/ArXiv-ML-Abstract-Embeddings/tree/main
    'juancopi81--yannic': ['embedding'], # https://huggingface.co/datasets/juancopi81/yannic_ada_embeddings/tree/main/data
    'Multimodal-Fatima--VQAv2': ['embedding'], # https://huggingface.co/datasets/Multimodal-Fatima/VQAv2_sample_test_embeddings/tree/main
    'Manzorq--hf_spsaces': ['embedding'], # https://huggingface.co/datasets/anzorq/hf-spaces-descriptions-embeddings
    'nickmuchi--netflix': ['embedding'], # https://huggingface.co/datasets/nickmuchi/netflix-shows-mpnet-embeddings
    'mitermix--mj5_clip': ['embedding'] #https://huggingface.co/datasets/mitermix/mj5-clip-l-14-oai-embeddings/tree/main
}

In [13]:
os.system('rm -f outputs/stats.json')
output_stats = {}
for fname, embed_name in file_to_embed_col.items():
    embed_name = embed_name[0]
    embed_name = 'embedding'
    table = pq.read_table(f'hug_data_embed_only/{fname}.zstd.parquet', columns=[embed_name])
    
    cur = table.column(embed_name)
    np.save(f'hug_data_embed_only/{fname}.v.npy', np.vstack(cur.to_numpy())) 
    np.save(f'hug_data_embed_only/{fname}.npy', cur.to_numpy()) 
    output_stats[f'numpy'] = get_size(f'hug_data_embed_only/{fname}.npy')
    output_stats[f'numpy-v'] = get_size(f'hug_data_embed_only/{fname}.v.npy')
    # print(type(cur.type))
    while type(cur[0].type) == pa.lib.ListType and type(cur[0][0].type) == pa.lib.ListType:
        cur = pa.compute.list_flatten(cur)
    embedding_table = pa.table([cur], names=['embedding'])
    pq.write_table(embedding_table, f'hug_data_embed_only/{fname}.zstd.parquet', compression='zstd')
    output_stats[f'parquet-zstd'] = get_size(f'hug_data_embed_only/{fname}.zstd.parquet')
    
    
    if cur[0][0].type == pa.uint8():
        cur = cur.cast(pa.list_(pa.int32()))
    embedding_table = pa.table([cur], names=['embedding'])
    po.write_table(embedding_table, f'hug_data_embed_only/{fname}.zstd.orc', compression='zstd')
    output_stats[f'orc-zstd'] = get_size(f'hug_data_embed_only/{fname}.zstd.orc')
    
    df = embedding_table.to_pandas()
    # Convert lists in df to numpy arrays and pad them to have the same length
    max_length = df['embedding'].str.len().max()
    # df['embedding'] = df['embedding'].apply(lambda x: np.pad(x, (0, max_length - len(x))))
    with h5py.File(f'hug_data_embed_only/{fname}.gzip.h5', 'w') as f:
        # For each column, create a dataset and save the numpy arrays
        for column in df.columns:
            # Create a fixed length dataset with GZIP compression
            ds = f.create_dataset(column, (len(df), max_length), dtype='float64', compression='gzip')
            # Save the numpy arrays to the dataset
            ds[:] = np.vstack(df[column].values)
    output_stats['fname'] = fname
    output_stats['hdf5-gzip'] = get_size(f'hug_data_embed_only/{fname}.gzip.h5')
    
    # Convert pandas dataframe to xarray dataset
    ds = xr.Dataset.from_dataframe(df)
    # Convert the DataFrame to a 2D numpy array
    np_array = np.stack(df['embedding'].values)
    # Create a multi-dimensional xarray DataArray
    da = xr.DataArray(np_array, dims=['x', 'y'])
    # if da's type is uint8, convert it to int8
    if da.dtype == 'uint8':
        da = da.astype('int8')
    for c in ['zstd']:
        compressor = zarr.Blosc(cname=c)
        # Write to Zarr format
        da.to_dataset(name='embedding').to_zarr(f'hug_data_embed_only/{fname}.{c}.zarr', encoding={"embedding": {"compressor": compressor}}, mode='w')
        z = zarr.open(f'hug_data_embed_only/{fname}.pqchunk.{c}.zarr', shape=np_array.shape, chunks=(65536, None), compressor=zarr.Blosc(cname=c), dtype=np_array.dtype)
        z[:] = np_array
        da.to_dataset(name='embedding').to_zarr(f'hug_data_embed_only/{fname}.{c}-level1.zarr', encoding={"embedding": {"compressor": zarr.Blosc(cname=c, clevel=1)}}, mode='w')
        da.to_dataset(name='embedding').to_zarr(f'hug_data_embed_only/{fname}.{c}-level1-bitshuffle.zarr', encoding={"embedding": {"compressor": zarr.Blosc(cname=c, clevel=1, shuffle=2)}}, mode='w')
        output_stats[f'zarr-{c}'] = get_size(f'hug_data_embed_only/{fname}.{c}.zarr')
        output_stats[f'zarr-{c}-level1'] = get_size(f'hug_data_embed_only/{fname}.{c}-level1.zarr')
        output_stats[f'zarr-{c}-level1-bitshuffle'] = get_size(f'hug_data_embed_only/{fname}.{c}-level1-bitshuffle.zarr')
        output_stats[f'zarr-{c}-pqchunk'] = get_size(f'hug_data_embed_only/{fname}.pqchunk.{c}.zarr')
    parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv outputs/{}_{}.csv'.format('hpc_file', timestamp))

0

In [10]:
os.system('send_email hdf5_test')

0